### Compilation/Generation Benchmark:

#### Requirements for running:
- ZoKrates installed (via install script)


#### Select files:
- Drop all files that are supposed to be benchmarked in the files/ directory in this project

#### Python Imports:

In [2]:
%%capture
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install csv2md
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import datetime
import statistics

#### Iterate Files:

##### Benchmark Settings:
- Each programm will be run N * R times
    - N: Number of loop iterations. The minimum value of all loop execution times is returned to eliminate side-effects from parallel processes on the benchmarking machine.
    - R: Number of repetitions in one loop. Values are averaged, stddev is calculated.

In [ ]:
compile_opt = []
compile_opt_stdev = []
constraints_opt = []
mem_usg = []
mem_usg_stdev = []
files = []
started = 0

# Set N and R here, also set in alias_magic!!
n = 5
r = 3
%alias_magic benchmark timeit -p "-n 1 -r 1 -o"

def compile_file(file, unopt):
    cmd = f"./memusg.sh zokrates compile -i files/{file} --light > constraints.txt"
    value = %benchmark !{cmd}
    compile_opt.append(int(value.best * 1000000))
    get_memusg()

# counts constraints by looking into out.ztf
def count_constraints(unopt):
    f = open("./constraints.txt",'r')
    lines = f.read().splitlines()
    length = lines[-1].split("Number of constraints: ")[1]
    if unopt:
        constraints_unopt.append(length)
    else:
        constraints_opt.append(length)
    
    
def get_memusg():
    with open('exports/data/memusg_res.txt', 'r+') as f:
        val_list = [int(line.rstrip()) for line in f]
        chunks = [val_list[x:x+r] for x in range(0, len(val_list), r)] # turns list into list of repetitions results -> dim: [n][r]
        mem_usg.append(min([statistics.mean(chunk) for chunk in chunks])) # takes average of each loop and selects min as result
        f.truncate(0)

# empties files from previous results
def reset_files():
    results = open("exports/data/result.csv", "w")
    results.write("file, compile_opt_microsec, memusg_compile_KiB, constraints\n")
    open("exports/data/memusg_res.txt", 'w').close()
    open("console_log.txt", 'w').close()
    
    
    
def export_data():
    file = open("exports/data/result.csv", "a")
    for i, val in enumerate(compile_opt):
        file.write(
            files[i] + ", " +
            str(val) + ", " + 
            str(mem_usg[i]) + ", " + 
            str(constraints_opt[i]) + "\n"
        )
    file.close()
    
reset_files()  
for file in sorted(os.listdir('./files')):
    if file.endswith(".zok"):
        print(file)
        started = datetime.datetime.now() 
        print("Started Optimized: " + started.strftime("%H:%M:%S") + "\n")
        print("Compiling: ", end =" ")
        compile_file(file, False)
        print("\nRan: " + str(datetime.datetime.now() - started))
        count_constraints(False)
        print("_________________________________________________\n")
    else:
        continue
    
    files.append(file.split('.')[0])

export_data()

In [1]:
!csv2md exports/data/result.csv > exports/data/table.md

with open("exports/data/table.md") as f:
    print(f.read())

| file |  compile_opt_microsec |  memusg_compile_KiB |  constraints |
| ---- | --------------------- | ------------------- | ------------ |



#### Compilation and Setup Diagram:


#### Witness and Proof Diagram: